In [ ]:

#%%capture
#!git clone https://github.com/ynop/py-ctc-decode
#!pip3 install transformers datasets torchaudio
#!pip3 install https://github.com/kpu/kenlm/archive/master.zip

In [3]:
#!git clone https://github.com/ynop/py-ctc-decode

fatal: destination path 'py-ctc-decode' already exists and is not an empty directory.


In [ ]:
#!pip install transformers datasets torchaudio

In [ ]:
#!pip install https://github.com/kpu/kenlm/archive/master.zip

In [25]:
#%cd py-ctc-decode

/srv/scratch/z5313567/thesis/JupyterLab/Language_model/py-ctc-decode


In [ ]:
# ##########################################################################################################
# ##########################################################################################################
#
# run the following command in the environment in Putty terminal
# !pip install https://github.com/kpu/kenlm/archive/master.zip
#
# ##########################################################################################################
# ##########################################################################################################

In [2]:
cd /srv/scratch/z5313567/thesis/JupyterLab/Language_model

/srv/scratch/z5313567/thesis/JupyterLab/Language_model


In [1]:
import librosa
import numpy as np

In [2]:
data_train_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/spontaneous/AusKidTalk_spontaneous_test_only_transcription_filepath.csv'
data_test_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/spontaneous/AusKidTalk_spontaneous_test_only_transcription_filepath.csv'
cache_fp = '/srv/scratch/z5313567/thesis/cache'
#model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/finetune_20230708_2'
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718'

# 1. Decoding audio data with Wav2Vec2 and a language model

In [3]:
from datasets import load_dataset

dataset = load_dataset('csv', 
                        data_files={'train': data_train_fp,
                                'test': data_test_fp},
                        cache_dir = cache_fp)

/home/z5313567/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/srv/scratch/z5313567/thesis/cache/csv/default-c29da4653c6396a5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 259.22it/s]


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['filepath', 'transcription_clean'],
        num_rows: 24
    })
    test: Dataset({
        features: ['filepath', 'transcription_clean'],
        num_rows: 24
    })
})

In [5]:
def speech_file_to_array_fn(batch):
    #speech_array, sampling_rate = sf.read(batch["filepath"])
    speech_array, sampling_rate = librosa.load(batch['filepath'], sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription_clean"]
    return batch

data = dataset.map(speech_file_to_array_fn, remove_columns=dataset.column_names["train"], num_proc=4)

Loading cached processed dataset at /srv/scratch/z5313567/thesis/cache/csv/default-c29da4653c6396a5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-4fdfbd2fb3c44f62_*_of_00004.arrow
Loading cached processed dataset at /srv/scratch/z5313567/thesis/cache/csv/default-c29da4653c6396a5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-10d51bc8e971ba36_*_of_00004.arrow


In [6]:
data

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 24
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 24
    })
})

In [7]:
import IPython.display as ipd

audio_sample = data["test"][9]
print(audio_sample["target_text"].lower())
ipd.Audio(data=audio_sample["speech"], autoplay=True, rate=audio_sample["sampling_rate"])

he was touching the dinosaur


In [8]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained(model_fp, cache_dir = cache_fp)
model = Wav2Vec2ForCTC.from_pretrained(model_fp, cache_dir = cache_fp)
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h", cache_dir = cache_fp)
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h",  cache_dir = cache_fp)
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-100h", cache_dir = cache_fp)
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-100h", cache_dir = cache_fp)

In [9]:
inputs = processor(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")

In [10]:
import torch

with torch.no_grad():
    logits = model(**inputs).logits

In [11]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

transcription

'HEY WASCOCKING THE DINOSAUR'

In [12]:
from transformers import Wav2Vec2ProcessorWithLM

processor1 = Wav2Vec2ProcessorWithLM.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm", cache_dir = cache_fp)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 155.06it/s]


In [13]:
logits.shape

torch.Size([1, 133, 32])

In [14]:
" ".join(sorted(processor1.tokenizer.get_vocab()))

"' </s> <pad> <s> <unk> A B C D E F G H I J K L M N O P Q R S T U V W X Y Z |"

In [15]:
transcription = processor1.batch_decode(logits.numpy()).text[0]
#transcription[0].lower()
transcription

'HE WAS COCKING THE DINOSAUR'

# 2. Getting data for the language model

In [4]:
full_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/storytelling.csv'

In [5]:
from datasets import load_dataset

dataset = load_dataset('csv', 
                        data_files = full_fp,
                        cache_dir = cache_fp)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 221.44it/s]


Dataset csv downloaded and prepared to /srv/scratch/z5313567/thesis/cache/csv/default-aef7eb02eb02bbac/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 42.58it/s]


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['transcription'],
        num_rows: 27
    })
})

In [7]:
chars_to_ignore_regex = '[,?.!*\-\;\:\"“%‘”�—’…–]'

In [13]:
import re

def extract_text(batch):
    text = batch["transcription"]
    batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
    return batch

In [14]:
dataset = dataset.map(extract_text, remove_columns=dataset.column_names['train'])
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 27
    })
})

# 3. Build an n-gram with KenLM

In [15]:
# ################################################################################################################################################################################
# ################################################################################################################################################################################
#
# Do not run the following command, since those dependencies are available as loadable modules on Katana:
# !sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
#
# Instead run the following commands in Putty terminal:
# 1) module load boost/1.80.0 eigen/3.4.0 zlib/1.2.13 bzip2/1.0.6 lzma/4.32.7
# 2) (can run this in JupyterLab notebook, never tried to run this in Putty termianl) ----> !wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
# 3) !mkdir kenlm/build -p && cd kenlm/build && cmake .. && make -j2
#
# ################################################################################################################################################################################
# ################################################################################################################################################################################

In [16]:
#!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

In [17]:
#!mkdir kenlm/build -p && cd kenlm/build && cmake .. && make -j2
#!ls kenlm/build/bin

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 27
    })
})

In [19]:
with open("text.txt", "w") as file:
    file.write(" ".join(dataset["train"]["text"]))

In [139]:
'''
def remove_patterns_from_text(text):
    # remove all the texts inside (), {}, []
    cleaned_text = re.sub(r"\[.*?\]|\(.*?\)|\{.*?\}", "", text)
    return cleaned_text

# Read the text from the text.txt
with open("text.txt", "r") as file:
    text = file.read()

# Clean the text by removing [xxx], (xxx), and {xxx}
cleaned_text = remove_patterns_from_text(text)
cleaned_text = cleaned_text.replace("{", "").replace("]", "")

# Writing cleaned text back into text.txt
with open("text.txt", "w") as file:
    file.write(cleaned_text)
    '''

In [20]:
# ################################################################################################################################################################################
# ################################################################################################################################################################################
#
# run the following commands in Putty terminal
# 1) cd /srv/scratch/z5313567/thesis/JupyterLab/Language_model
# 2) kenlm/build/bin/lmplz -o 5 --verbose header --text text.txt --arpa 5gram.arpa --discount_fallback (Reference: https://blog.csdn.net/w5688414/article/details/106994706)
#    or !kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"  
#
# ################################################################################################################################################################################
# ################################################################################################################################################################################


#!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"

In [21]:
!head -20 5gram.arpa

\data\
ngram 1=58341
ngram 2=890392
ngram 3=2389243
ngram 4=3339808
ngram 5=3642375

\1-grams:
-5.9495034	<unk>	0
0	<s>	-0.12275371
-2.1739275	a	-0.68441576
-3.5523458	green	-0.3070794
-5.8102064	skined	-0.12275371
-4.0021734	baby	-0.26539335
-5.2018	hulk	-0.12275371
-2.4619346	is	-0.6397725
-3.6977127	holding	-0.62519646
-4.262992	bottle	-0.31532252
-1.9762874	of	-0.71516836
-3.973714	milk	-0.36838007


In [23]:
#  The 5-gram correctly includes a "Unknown" or <unk>, as well as a begin-of-sentence, <s> token, but no end-of-sentence, </s> token. 
#  This sadly has to be corrected currently after the build.
# We can simply add the end-of-sentence token by adding the line 0 </s> -0.09522562 below the begin-of-sentence token and increasing the ngram 1 count by 1.
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
    has_added_eos = False
    for line in read_file:
        if not has_added_eos and "ngram 1=" in line:
            count=line.strip().split("=")[-1]
            write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
        elif not has_added_eos and "<s>" in line:
            write_file.write(line)
            write_file.write(line.replace("<s>", "</s>"))
            has_added_eos = True
        else:
            write_file.write(line)

In [24]:
!head -20 5gram_correct.arpa

\data\
ngram 1=58342
ngram 2=890392
ngram 3=2389243
ngram 4=3339808
ngram 5=3642375

\1-grams:
-5.9495034	<unk>	0
0	<s>	-0.12275371
0	</s>	-0.12275371
-2.1739275	a	-0.68441576
-3.5523458	green	-0.3070794
-5.8102064	skined	-0.12275371
-4.0021734	baby	-0.26539335
-5.2018	hulk	-0.12275371
-2.4619346	is	-0.6397725
-3.6977127	holding	-0.62519646
-4.262992	bottle	-0.31532252
-1.9762874	of	-0.71516836


# 4. Combine an n-gram with Wav2Vec2

In [5]:
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718'

In [6]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(model_fp)

/home/z5313567/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [8]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /srv/scratch/z5313567/thesis/JupyterLab/Language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


In [9]:
(list(sorted_vocab_dict.keys()))

['<pad>',
 '<s>',
 '</s>',
 '<unk>',
 '|',
 'E',
 'T',
 'A',
 'O',
 'N',
 'I',
 'H',
 'S',
 'R',
 'D',
 'L',
 'U',
 'M',
 'W',
 'C',
 'F',
 'G',
 'Y',
 'P',
 'B',
 'V',
 'K',
 "'",
 'X',
 'J',
 'Q',
 'Z']

In [10]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [12]:
processor_with_lm

Wav2Vec2ProcessorWithLM:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718', vocab_size=32, model_max_length=9223372036854775807, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [14]:
lm_save_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3'

In [15]:
import shutil

# Copy the entire source_folder to the destination_folder
shutil.copytree(model_fp, lm_save_fp)

'/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3'

In [16]:
processor_with_lm.save_pretrained(lm_save_fp)

In [17]:
!kenlm/build/bin/build_binary /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3/language_model/5gram_correct.arpa /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3/language_model/5gram.bin

Reading /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [18]:
!rm /srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3/language_model/5gram_correct.arpa

# 5 Combining LM with pretrained LM 

In [48]:
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718'
LM_1 = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk_scripted_spontaneous_combined/finetune_20230718_with_lm_3'
LM_pretrained = "patrickvonplaten/wav2vec2-base-100h-with-lm"

In [49]:
processor = Wav2Vec2Processor.from_pretrained(model_fp, cache_dir = cache_fp)
model = Wav2Vec2ForCTC.from_pretrained(model_fp, cache_dir = cache_fp)

processor_LM_1 = Wav2Vec2ProcessorWithLM.from_pretrained(LM_1, cache_dir = cache_fp)
model_LM_1 = Wav2Vec2ForCTC.from_pretrained(LM_1, cache_dir = cache_fp)
    
processor_LM_pretrained = Wav2Vec2ProcessorWithLM.from_pretrained(LM_pretrained, cache_dir = cache_fp)
model_LM_pretrained = Wav2Vec2ForCTC.from_pretrained(LM_pretrained, cache_dir = cache_fp)
    

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 1625.07it/s]


In [50]:
audio_sample = data["test"][6]
print(audio_sample["target_text"].lower())
ipd.Audio(data=audio_sample["speech"], autoplay=True, rate=audio_sample["sampling_rate"])

hulk tries to hide from the dinosaur and is still very scared


In [51]:
weight_model_1 = 0.00005
weight_model_pretrained = 1-weight_model_1;

In [52]:
input_values_LM_1 = processor_LM_1(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")
#input_values_LM_1 = processor(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")
input_values_LM_pretrained = processor_LM_pretrained(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")

In [53]:
with torch.no_grad():
    logits_LM_1 = model_LM_1(**input_values_LM_1).logits
    logits_LM_pretrained = model_LM_pretrained(**input_values_LM_pretrained).logits

In [54]:
combined_logits = weight_model_1 * logits_LM_1 + weight_model_pretrained * logits_LM_pretrained

In [55]:
pred_ids_LM = torch.argmax(combined_logits, dim=-1)
transcription = processor.batch_decode(pred_ids_LM)[0]
transcription

'HOW CE TRIES TO HIDE FROM THE DINASOL AND IS STILL VERY SCARED'

In [56]:
# prediction using newly-created LM on the combined_logits
transcription = processor_LM_1.batch_decode(combined_logits.numpy()).text[0]
transcription

'HOWCETRIES TO HIDE FROM THEDINASOL AND IS STILL VERY SCARED'

In [57]:
# prediction using pretrained LM on the combined_logits
transcription = processor_LM_pretrained.batch_decode(combined_logits.numpy()).text[0]
transcription

'HOW HE TRIES TO HIDE FROM THE DINASOL AND IS STILL VERY SCARED'

In [58]:
# prediction using newly-created LM on its own logits
transcription = processor_LM_1.batch_decode(logits_LM_1.numpy()).text[0]
transcription

'HOW HETRISETOHIDE ROM THEDINOSAURAND ISTILEVERYSKIRT'

In [59]:
# prediction using pretrained LM on its own logits
transcription = processor_LM_pretrained.batch_decode(logits_LM_pretrained.numpy()).text[0]
transcription

'HOW HE TRIES TO HIDE FROM THE DINASOL AND IS STILL VERY SCARED'

In [ ]:
processor = Wav2Vec2ProcessorWithLM.from_pretrained(lm_save_fp, cache_dir = cache_fp)
processor1 = Wav2Vec2ProcessorWithLM.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm", cache_dir = cache_fp)


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 14939.64it/s]


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(lm_save_fp, cache_dir = cache_fp)
model1 = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm", cache_dir = cache_fp)

In [ ]:
audio_sample = data["test"][18]
print(audio_sample["target_text"].lower())
ipd.Audio(data=audio_sample["speech"], autoplay=True, rate=audio_sample["sampling_rate"])

the dinosaur was sad


In [ ]:
inputs = processor(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")
inputs1 = processor1(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")

In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits
    logits1 = model1(**inputs1).logits

In [ ]:
weight_model = 0.15
weight_model1 = 1-weight_model

In [ ]:
combined_logits = weight_model * logits + weight_model1 * logits1

In [44]:
#predicted_ids = torch.argmax(combined_logits, dim=-1)

In [45]:
transcription = processor.batch_decode(combined_logits.numpy()).text[0]
transcription

'THEDINOSAURWS'

In [46]:
transcription = processor1.batch_decode(combined_logits.numpy()).text[0]
transcription

'THE DINOSAUR WAS THAT'

In [47]:
transcription = processor.batch_decode(logits.numpy()).text[0]
transcription

'THEDINOSAURWAS AT'

In [48]:
transcription = processor1.batch_decode(logits1.numpy()).text[0]
transcription

'DEAD DYING THE SILL WAS THAT'

In [ ]:
processor2 = Wav2Vec2Processor.from_pretrained(model_fp)
pred_ids = torch.argmax(combined_logits, dim=-1)
transcription = processor2.batch_decode(pred_ids)[0]
transcription

'DED DN I SILL WA TAT'